<a href="https://colab.research.google.com/github/BBotond03/NASA_Space_Apps_Challenge_BHAF/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow


In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Machine learning and preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Keras (part of TensorFlow) for building the RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Set a random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# Read data
file_path = 'path_to_your_file.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the data
print(data.head())

# Select relevant features
X = data[['velocity']]  # Input features (velocity)
y = data['mq_type']     # Output labels (impact types)

# Label encode the impact types (mq_type)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encode the labels for classification
y_onehot = to_categorical(y_encoded)

# Normalize the velocity data (optional but recommended)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_onehot, test_size=0.2, random_state=random_seed)

# Reshape the input data to be compatible with RNN (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the RNN Model
model = Sequential()

# RNN Layer
model.add(SimpleRNN(50, input_shape=(1, X_train.shape[2]), activation='tanh'))

# Dense Layer with 200 neurons
model.add(Dense(200, activation='relu'))

# Dropout layer with 50% dropout rate
model.add(Dropout(0.5))

# Add a dense output layer with softmax for multi-class classification
model.add(Dense(y_onehot.shape[1], activation='softmax'))

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitors validation loss
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restores the weights of the best epoch after training
)

# Train the Model with 10% validation split
history = model.fit(X_train, y_train,
                    epochs=10000,  # Set a large number of epochs, training will stop early if no improvement
                    batch_size=32,
                    validation_split=0.1,  # 10% of the training data used for validation
                    callbacks=[early_stopping],  # Add EarlyStopping
                    verbose=1)

# Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the model in .keras format
model.save('model.keras')  # Save in .keras format

print("Model saved successfully as model.keras.")


KeyboardInterrupt: 